#### 2019/05/08
# Mask R-CNN 
- 注意源码中的size()可能需要改成size()[0]
## ideas
- [ ] mask rcnn的结果进行ppo, roialign或rpn之后的特征处开始训练actor critic, 看看最终能不能帮助改善rcnn, 主要是为了提高iou
- [ ] 球坐标经纬度范围加zoom信息及其归一化与置信度，经纬度上的高斯，随时间方差增大
- [ ] region proposal with hsv 记录邻域, 宽阈值筛选， 然后再精细化阈值， 辨认脸型
- [ ] 图像的小片方差也有它的边缘信息, 即, 两个预处理, LAB转换和方差map

## 今日任务
- [x] 跑通mask rcnn
- [ ] 了解[俄罗斯小哥](https://mp.weixin.qq.com/s/3B3YVGaIRAzEaVojD0Qq3g)的[github](https://github.com/qubvel/segmentation_models.pytorch)


## 以前任务
- [ ] 把annotation里面的游程编码一条一条在原图上标出来,搞懂,并且会用coco.py给的接口
- [ ] 搞明白评价接口,能随意使用判定预测结果的准确率
- [x] 开始找DQN pytorch的代码
- [ ] 开始找PPO pytorch的代码
- [ ] 找到rcnn训练崩溃的原因
- [ ] 训练一个神经网络来指导(为instance打分)选择对象以及调整, 考虑bbox大小, 中心位置占画面长宽的比例, 
- [ ] 确定一个默认的overview视角作为后备
- [ ] 设计一个机制,五级打分:没啥变化, 变好了, 变坏了, 太好了, 太坏了, 人只进行这样的评判, 然后可以在线地传递到instance的打分上
- [ ] 结合yolov3与pose body, 实现快速识别
- [ ] 自动提取object的skeleton信息，然后dqn之，以完善边缘的准确率
- [x] 开始探索zoo
- [ ] 赋予AI一些记忆(Memory),哪怕7秒的记忆,让前面一段时间识别的东西有留存,并且指导最新的工作.而不是摄像头的机械记忆.

- [x] 

## 工作


```shell 
python coco.py train --dataset=/home/xianr/data/utils/datasets/coco --model=xpfcoco
python coco.py train --dataset=/home/xianr/TurboProjects/coco --model=coco
python coco.py train --dataset=/home/xianr/TurboProjects/coco --model=last
```

## 报错: 
```python
/home/xianr/data/projects/pytorch-mask-rcnn/model.py(439)pyramid_roi_align()
-> y1, x1, y2, x2 = boxes.chunk(4, dim=1)
boxes = inputs[0]
inputs 来自 def pyramid_roi_align(inputs, pool_size, image_shape): (406)
```


## nn.Module的__setattr__是被重写了的，所以在self.rpn = RPN()的时候，会调用nn.Module的__setattr__，判断加进来的
- ### 是不是Parameter，若是，加到当前module的Parameter的name下
- ### 是不是Module，若是，加到当前module的name下 ```modules[name] = value```
- ### 变量名name在不在_buffer里，若在，更新_buffer的name ```buffers[name] = value```
- ### 若都不是, 走object的_setattr__, 也就是正常的self.name = value

##  coco评估时的转换
```Python
(Pdb) cocoEval._dts[(90155, 7)][0].keys()
    dict_keys(['image_id', 'category_id', 'bbox', 'score', 'segmentation', 'area', 'id', 'iscrowd'])
(Pdb) cocoEval._gts[(90155, 7)][0].keys()
    dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id', 'ignore', '_ignore'])
```
- 可见, groundtruth和detection的字典结构基本一样, 唯一的区别在于gt有'ignore'和'_ignore'而没有'score'

### How to create mask images from COCO dataset?
- [How to create mask images from COCO dataset?](https://stackoverflow.com/questions/50805634/how-to-create-mask-images-from-coco-dataset)
- code

```python
dataDir='G:'
dataType='train2014'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)


coco=COCO(annFile)
annFile = '{}/annotations/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)


catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds );
imgIds = coco.getImgIds(imgIds = imgIds[0])
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
I = io.imread('G:/train2014/'+img['file_name'])

plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

#######################################
mask = coco.annToMask(anns[0])
for i in range(len(anns)):
    mask += coco.annToMask(anns[i])

plt.imshow(mask)
```